In [1]:
# 21CS30035 : Paramananda Bhaskar
# [ Project Code: IMHC-AC ]
# Interest Match using Complete Linkage Agglomerative (Bottom-Up) Clustering Technique

In [2]:
import csv
import math
import random
from itertools import combinations

def load_data(filename):
    data = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            # Filter out empty cells and non-integer values
            filtered_row = [int(x) for x in row if x.strip().isnumeric()]
            if filtered_row:
                data.append(filtered_row)
    
    # Normalize the data
    min_vals = [min(row) for row in zip(*data)]
    max_vals = [max(row) for row in zip(*data)]
    normalized_data = [[(x - min_val) / (max_val - min_val) for x, min_val, max_val in zip(row, min_vals, max_vals)] for row in data]

#     # Z-score normalization
#     means = [sum(row) / len(row) for row in zip(*data)]
#     std_devs = [math.sqrt(sum((x - mean) ** 2 for x in row) / len(row)) for row, mean in zip(zip(*data), means)]
#     normalized_data = [[(x - mean) / std_dev if std_dev != 0 else 0 for x, mean, std_dev in zip(row, means, std_devs)] for row in data]
    
#     return normalized_data
    return data


# Cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    dot_product = sum(a * b for a, b in zip(vec1, vec2))
    magnitude1 = math.sqrt(sum(a ** 2 for a in vec1))
    magnitude2 = math.sqrt(sum(b ** 2 for b in vec2))
    return dot_product / (magnitude1 * magnitude2)

# Initialize cluster centroids randomly
def initialize_centroids(data, k):
    centroids = random.sample(data, k)
    return centroids

# Assign each data point to the nearest centroid
def assign_clusters(data, centroids):
    clusters = [[] for _ in range(len(centroids))]
    for i, point in enumerate(data):
        nearest_centroid_index = min(range(len(centroids)), key=lambda j: (1-cosine_similarity(point, centroids[j])))
        clusters[nearest_centroid_index].append(i)
    return clusters

# Update centroids based on mean of data points in each cluster
def update_centroids(data, clusters):
    centroids = []
    for cluster in clusters:
        cluster_points = [data[i] for i in cluster]
        centroid = [sum(col) / len(cluster_points) for col in zip(*cluster_points)]
        centroids.append(centroid)
    return centroids

# Perform K-means clustering
def kmeans(data, k, max_iterations=20):
    centroids = initialize_centroids(data, k)
    for _ in range(max_iterations):
        clusters = assign_clusters(data, centroids)
        new_centroids = update_centroids(data, clusters)
        if new_centroids == centroids:
            break
        centroids = new_centroids
    return clusters

# Calculate silhouette coefficient for a single data point
def silhouette_coefficient(data, clusters, point_index):
    cluster_index = None
    for i, cluster in enumerate(clusters):
        if point_index in cluster:
            cluster_index = i
            break
    a = 0  # Mean distance to other points in the same cluster
    b = float('inf')  # Mean distance to points in the nearest cluster
    for i, cluster in enumerate(clusters):
        if i == cluster_index:
            for j in cluster:
                if j != point_index:
                    a += (1-cosine_similarity(data[point_index], data[j]))
            if len(cluster) > 1:
                a /= (len(cluster) - 1)
        else:
            mean_distance = 0
            for j in cluster:
                mean_distance += (1-cosine_similarity(data[point_index], data[j]))
            mean_distance /= len(cluster)
            b = min(b, mean_distance)
    return (b - a) / max(a, b)

# Calculate silhouette score for all data points
def silhouette_score(data, clusters):
    total_score = 0
    for i in range(len(data)):
        total_score += silhouette_coefficient(data, clusters, i)
    return total_score / len(data)

# Write cluster information to file
def write_clusters_to_file(clusters, filename):
    with open(filename, 'w') as file:
        for cluster in sorted(clusters, key=lambda x: min(x)):
            file.write(','.join(map(str, sorted(cluster))))
            file.write('\n')

In [3]:
def pairwise_similarity(data):
    n = len(data)
    similarity_matrix = [[0] * n for _ in range(n)]
    for i, j in combinations(range(n), 2):
        similarity_matrix[i][j] = cosine_similarity(data[i], data[j])
        similarity_matrix[j][i] = similarity_matrix[i][j]
    return similarity_matrix

def find_closest_clusters(similarity_matrix, clusters):
    min_similarity = float('inf')
    closest_clusters = None
    n = len(similarity_matrix)
    for i, j in combinations(range(n), 2):
        if (i in clusters) and (j in clusters):
            if similarity_matrix[i][j] < min_similarity:
                min_similarity = similarity_matrix[i][j]
                closest_clusters = (i, j)
    return closest_clusters

def complete_linkage(data, k):
    similarity_matrix = pairwise_similarity(data)
#     print(similarity_matrix)

    print("\nStarting Complete Linkage Agglomerative (Bottom-Up) Clustering algorithm...")
    clusters = [{i} for i in range(len(data))]  # Initialize each point as a separate cluster
    print(f"Starting with {len(clusters)} clusters")
    
    while len(clusters) > k:
        i, j = find_closest_clusters(similarity_matrix, set(range(len(clusters))))
        clusters[i] |= clusters[j]  # Merge clusters i and j
        clusters.pop(j)  # Remove cluster j
        # Update similarity matrix
        for m in range(len(clusters)):
            if m != i and m != j:
                similarity_matrix[i][m] = max(similarity_matrix[i][m], similarity_matrix[j][m])
                similarity_matrix[m][i] = similarity_matrix[i][m]
        print(f"Currently {len(clusters)} clusters")  # Print the current number of clusters
    return clusters

def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def map_clusters(kmeans_clusters, hierarchical_clusters):
    jaccard_scores = []
    for kmeans_cluster in kmeans_clusters:
        max_similarity = float('-inf')
        best_match = None
        for hierarchical_cluster in hierarchical_clusters:
            similarity = jaccard_similarity(set(kmeans_cluster), set(hierarchical_cluster))
            if similarity > max_similarity:
                max_similarity = similarity
                best_match = hierarchical_cluster
        jaccard_scores.append(max_similarity)
    return jaccard_scores

In [4]:
def main():
    # Step 1: K-means clustering with k=3
    data = load_data('interests.csv')

    # Step 2: Evaluate clustering using Silhouette coefficient and Find optimal value of k
    optimal_k = None
    max_silhouette = float('-inf')
    for k in range(3, 7):
        clusters = kmeans(data, k)
        silhouette = silhouette_score(data, clusters)
        print(f"Silhouette coefficient in k-means Clustering for k={k}:", silhouette)
        if silhouette > max_silhouette:
            max_silhouette = silhouette
            optimal_k = k

    print("\nOptimal value of k:", optimal_k)

    # Step 3: Write final cluster information to file using optimal k
    clusters_optimal = kmeans(data, optimal_k)
    print("\nOptimal k-means Cluster:")
    print(clusters_optimal)
    write_clusters_to_file(clusters_optimal, 'kmeans.txt')

    # Step 4: Hierarchical clustering with complete linkage
    hierarchical_clusters = complete_linkage(data, optimal_k)
    print("\nOptimal Hierarchical Cluster:")
    print(hierarchical_clusters)
    silhouette = silhouette_score(data, hierarchical_clusters)
    write_clusters_to_file(hierarchical_clusters, 'agglomerative.txt')

    # Step 5: Map clusters and calculate Jaccard similarity
    jaccard_scores = map_clusters(clusters_optimal, hierarchical_clusters)
    for i, score in enumerate(jaccard_scores):
        print(f"Jaccard similarity for cluster {i+1}: {score}")

if __name__ == "__main__":
    main()

Silhouette coefficient in k-means Clustering for k=3: 0.19713028527954768
Silhouette coefficient in k-means Clustering for k=4: 0.1252956263981666
Silhouette coefficient in k-means Clustering for k=5: 0.0894102836793119
Silhouette coefficient in k-means Clustering for k=6: 0.1487045293331698

Optimal value of k: 3

Optimal k-means Cluster:
[[0, 2, 7, 9, 11, 12, 13, 17, 18, 22, 23, 24, 25, 26, 29, 33, 35, 38, 41, 43, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 57, 60, 67, 69, 77, 78, 84, 86, 87, 91, 97, 98, 99, 102, 103, 107, 110, 111, 112, 115, 120, 123, 124, 125, 128, 130, 133, 136, 138, 139, 141, 142, 144, 148, 151, 152, 155, 158, 159, 164, 165, 166, 168, 170, 171, 172, 174, 177, 179, 182, 183, 185, 186, 188, 189, 190, 191, 192, 193, 196, 201, 203, 205, 206, 207, 208, 210, 211, 215, 216, 217, 223, 226, 228, 230, 231, 233, 236, 239, 242, 244, 245, 247, 248, 249, 250, 251, 252, 255, 259, 260, 261, 266, 267, 268, 270, 273, 274, 278, 280, 287, 288, 290, 292, 294, 295, 298, 300, 302, 303, 304